# Credit Card Fraud Detection

This notebook implements a machine learning solution for detecting fraudulent credit card transactions. The dataset contains anonymized features obtained through PCA, along with transaction time and amount. Due to the highly imbalanced nature of the data, special attention is given to evaluation metrics such as precision and recall.

The logistic regression, Decision Tree Classifier and Random Forest Classifier models are used with techniques like class weighting to handle imbalance and ensure robust fraud detection.

[Download Dataset from Google Drive](https://drive.google.com/file/d/1N-ReBFzEEasS9wrGl0PjPKZQuQUGSKFD/view?usp=drive_link)


In [2]:
# Importing the depedencies

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the dataset and some basic exploration:

data = pd.read_csv('creditcard.csv')
data.head(3)
print('-'*100)
print(data.info())
print("-" * 100)
print(data.describe())
print("-" * 100)
print(data.shape)
print("-" * 100)
print(data.columns)
print("-" * 100)
print(data.isnull().sum())
print("-" * 100)
print(data.duplicated().sum())

In [ ]:
 # Checking the distribution of legit vs fraud transactions

data['Class'].value_counts() # This dataset is highly unbalanced.

Class
0    284315
1       492
Name: count, dtype: int64

In [16]:
# Separating the normal and fraud transactions:

legit = data[data.Class == 0]
fraud = data[data.Class == 1]
print(legit.shape)
print(fraud.shape)


(284315, 31)
(492, 31)


In [18]:
# Statistical measures of the data:

print(legit.Amount.describe())
print("-" * 100)
print(fraud.Amount.describe())
print("-" * 100)

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64
----------------------------------------------------------------------------------------------------
count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64
----------------------------------------------------------------------------------------------------


In [20]:
# Comparing the values for both transactions:

data.groupby('Class').mean() 

# Groupby function is used to group the data based on a particular column and then we can perform some aggregate functions on it.

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [ ]:
# Undersampling the dataset to balance the data:

# We cannot use the dataset as it is because it is highly unbalanced. So, we will create a new dataset with equal number of legit and fraud transactions.

legit_sample = legit.sample(n=492) # Taking a random sample of 492 legit transactions

balanced_data = pd.concat([legit_sample, fraud], axis =0) # axis = 0 means we are concatenating rows
print(balanced_data.shape)

print(balanced_data['Class'].value_counts()) 

(984, 31)
Class
0    492
1    492
Name: count, dtype: int64


In [25]:
balanced_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,97387.764228,0.024595,-0.039175,-0.027106,-0.133683,0.037159,0.016887,-0.001405,0.127715,0.014826,...,0.027498,0.021193,0.045549,-0.013617,0.052452,0.001986,0.032319,0.026943,0.000752,89.704045
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [27]:
# Splitting the data into features and targets:

X = balanced_data.drop(columns = 'Class', axis = 1)
Y = balanced_data['Class']

# print(X)
print(Y)

230938    0
76408     0
215592    0
97547     0
167612    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [28]:
# Splitting the data into training and testing data:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2, stratify= Y)
print(X.shape, X_train.shape, X_test.shape)


(984, 30) (787, 30) (197, 30)


In [30]:
# Training the models:

# Logistic Regression
model = LogisticRegression()
model.fit(X_train, Y_train)


LogisticRegression()

In [31]:
# Decision Tree Classifier:
model1 = DecisionTreeClassifier()
model1.fit(X_train, Y_train)

DecisionTreeClassifier()

In [32]:
# Random Forest Classifier:

model2 = RandomForestClassifier()
model2.fit(X_train, Y_train)

RandomForestClassifier()

In [43]:
# Model Evaluation: Logistic Regression:

# Accuracy Score:

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print(f'Accuracy score of training data : {training_data_accuracy*100:.2f}%')

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print(f'Accuracy score of test data : {test_data_accuracy*100:.2f}%')

print("-" * 100)

# Confusion Matrix:

cm = confusion_matrix(Y_test, X_test_prediction)
print('Confusion Matrix : \n', cm)

print("-"*100)

# Classification Report:
cr = classification_report(Y_test, X_test_prediction)
print('Classification Report : \n', cr)

Accuracy score of training data : 95.04%
Accuracy score of test data : 94.42%
----------------------------------------------------------------------------------------------------
Confusion Matrix : 
 [[97  2]
 [ 9 89]]
----------------------------------------------------------------------------------------------------
Classification Report : 
               precision    recall  f1-score   support

           0       0.92      0.98      0.95        99
           1       0.98      0.91      0.94        98

    accuracy                           0.94       197
   macro avg       0.95      0.94      0.94       197
weighted avg       0.95      0.94      0.94       197



In [44]:
# Model Evaluation: Random Forest Classifier:

# Accuracy Score:

X_train_prediction = model1.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print(f"Accuracy score of training data : {training_data_accuracy*100:.2f}%")

print("-" * 100)

X_test_prediction = model1.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print(f"Accuracy score of test data : {test_data_accuracy*100:.2f}%")

print("-" * 100)

# Confusion Matrix:
cm = confusion_matrix(Y_test, X_test_prediction)
print('Confusion Matrix : \n', cm)

print("-" * 100)

# Classification Report:
cr = classification_report(Y_test, X_test_prediction)
print('Classification Report : \n', cr)

Accuracy score of training data : 100.00%
----------------------------------------------------------------------------------------------------
Accuracy score of test data : 89.85%
----------------------------------------------------------------------------------------------------
Confusion Matrix : 
 [[91  8]
 [12 86]]
----------------------------------------------------------------------------------------------------
Classification Report : 
               precision    recall  f1-score   support

           0       0.88      0.92      0.90        99
           1       0.91      0.88      0.90        98

    accuracy                           0.90       197
   macro avg       0.90      0.90      0.90       197
weighted avg       0.90      0.90      0.90       197



In [45]:
# Model Evaluation: Decision Tree Classifier:

# Accuracy Score:

X_train_prediction = model2.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print(f"Accuracy score of training data : {training_data_accuracy*100:.2f}%")

X_test_prediction = model2.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print(f"Accuracy score of test data : {test_data_accuracy*100:.2f}%")

print("-" * 100)

# Confusion Matrix:
cm = confusion_matrix(Y_test, X_test_prediction)
print("Confusion Matrix : \n", cm)

print("-" * 100)

# Classification Report:
cr = classification_report(Y_test, X_test_prediction)
print("Classification Report : \n", cr)

Accuracy score of training data : 100.00%
Accuracy score of test data : 92.39%
----------------------------------------------------------------------------------------------------
Confusion Matrix : 
 [[97  2]
 [13 85]]
----------------------------------------------------------------------------------------------------
Classification Report : 
               precision    recall  f1-score   support

           0       0.88      0.98      0.93        99
           1       0.98      0.87      0.92        98

    accuracy                           0.92       197
   macro avg       0.93      0.92      0.92       197
weighted avg       0.93      0.92      0.92       197

